### Code to visualise training and performance of Learn Transform.ipynb

#### Imports and parameter definitions

In [ ]:
import os
import sys
sys.path.insert(0, os.path.abspath("../.."))

import torch
import matplotlib.pyplot as plt

import autoencoders
import learning_plotters as lp
import Double_Pendulum.robot_parameters as robot_parameters
import Double_Pendulum.dynamics as dynamics
import training_data as training_data

from torch.utils.data import TensorDataset, Dataset, DataLoader

import matplotlib
matplotlib.rcParams['font.family']   = 'serif'
matplotlib.rcParams['font.serif']    = ['Times New Roman']
matplotlib.rcParams['mathtext.fontset'] = 'dejavuserif'

%load_ext autoreload
%autoreload 2

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

rp = robot_parameters.LUMPED_PARAMETERS.copy()
rp["m0"] = 0.0

In [ ]:
file_counter = 0
train_clockwise = False

#### Load models

In [ ]:
dir_name = "NN_full-q"
#dir_name = "NN_202505141642(half-q)"

dir_path = os.path.join(os.getcwd(), "Models", dir_name)
nn_filename = "NN" + "_0.pth"
nn_filepath = os.path.join(dir_path, nn_filename)

model_path = nn_filepath
model = autoencoders.Autoencoder_double(rp).to(device)  # Initialize model architecture
model.load_state_dict(torch.load(model_path, weights_only=True, map_location=device))  # Load weights

model_ana = autoencoders.Analytic_transformer(rp)

models = [model_ana, model]
model_names = ["Analytic", "Learned"]

In [ ]:
def load_loss(load_loss_path):
	losses_dict = torch.load(load_loss_path, map_location = "cpu", weights_only=True)
	return losses_dict["train"], losses_dict["val"]

#### Visualize loss plot, single model

In [ ]:
single_losses_path = "Models/" + dir_name + "/losses.pt"
train_loss, val_loss = load_loss(single_losses_path)

In [ ]:
lp.plot_loss(train_loss, val_loss, file_counter, log = True, save_folder = dir_path)

In [ ]:
def mask_points(q0_split, clockwise = False, q1_margin = 0.):

	"""
	Returns a set of [q0, q1] points based on "q0_split" limits on q0 and q1.
	The limits on q1 depend on whether a clockwise or counterclockwise dataset is selected.
	"""   

	# Retrieve training points
	points = training_data.points.to(device)
	
	# Mask to retrieve only the counterclockwise points
	width_mask = (points[:,0] >= q0_split[0]) & (points[:,0] <= q0_split[1])
	ccw_mask = ((points[:,1] >= points[:,0] + q1_margin) & 
				  (points[:,1] <= points[:,0] + torch.pi - q1_margin))
	
	# Mask to retrieve only the clockwise points
	cw_mask = ((points[:,1] >= points[:,0] - torch.pi + q1_margin) & (points[:,1] <= points[:,0] - q1_margin))

	if clockwise:
		final_mask = width_mask & cw_mask
	else:
		final_mask = width_mask & ccw_mask
	
	points = points[final_mask]
	points = points[0:6000]

	if points.size(0) < 6000:
		print("Warning: Only", points.size(0), "points in dataset.")

	return(points)

In [ ]:
def make_dataset(points):

	"""
	Compute mass- and input matrix of all training points to reduce load in training.
	Returns TensorDataset of (q, M_q, A_q). 
	"""

	data_pairs = []
	for point in points:
		Mq_point, _, _ = dynamics.dynamical_matrices(rp, point, point)
		Aq_point = dynamics.input_matrix(rp, point)
		data_pairs.append((point, Mq_point, Aq_point))

	points_tensor = torch.stack([pair[0] for pair in data_pairs])           # Tensor of all points
	mass_matrices_tensor = torch.stack([pair[1] for pair in data_pairs])   # Tensor of all mass matrices
	input_matrices_tensor = torch.stack([pair[2] for pair in data_pairs])  # Tensor of all input matrices

	# Create TensorDataset
	dataset = TensorDataset(points_tensor, mass_matrices_tensor, input_matrices_tensor)
	return(dataset)


In [ ]:
def make_plot_dataloader(dataset, stride = 1):

	"""
	Takes the training dataset and returns a dataloader of every 10th point
	to reduce visual clutter. 
	"""

	points_tensor, mass_matrices_tensor, input_matrices_tensor = dataset.tensors
	
	plot_sampled = points_tensor[::stride]
	mass_sampled = mass_matrices_tensor[::stride]
	input_sampled = input_matrices_tensor[::stride]

	plot_dataset = TensorDataset(plot_sampled, mass_sampled, input_sampled)
	plot_dataloader = DataLoader(plot_dataset, batch_size=len(plot_dataset), shuffle=False, num_workers=0)

	return(plot_dataloader)

In [ ]:
q0_split = (-torch.pi, torch.pi)
q1_margin = 0.2

plt.ion()
plot_points = mask_points(q0_split, clockwise = train_clockwise, q1_margin = q1_margin)
plot_dataset = make_dataset(plot_points)
plot_dataloader = make_plot_dataloader(plot_dataset, stride = 1)

In [ ]:
lp.plot_model_performance(model, model_ana, plot_dataloader, dir_path, device)

#### Plot loss for multiple models

In [ ]:
loss_paths = [
    "Models/NN_202505221832/losses.pt",
    "Models/NN_202505221839/losses.pt",
    "Models/NN_202505221919/losses.pt"
]

In [ ]:
train_losses = []
val_losses   = []

for loss_path in loss_paths:
	train_loss, val_loss = load_loss(loss_path)
	train_losses.append(train_loss)
	val_losses.append(val_loss)

In [ ]:
lp.plot_losses_vs_epoch(train_losses, val_losses, save_folder = dir_path)

#### Plot Yin-Yang for $\theta$ vs $x, y$

In [ ]:
# Define the number of grid points along each dimension.
n_points = 200

lp.plot_yinyang(n_points, q0_split, dir_path, file_counter, train_clockwise, models, model_names, rp, device) #TODO: Fill in